In [1]:
import json
import os

In [9]:
import os

current = '/Users/jinym/Downloads/잡지_이미지제외/'

# for root, dirs, files in os.walk(current):
    # for file in files:
        # os.system(f'unzip {os.path.join(root,file)} && mv crop /Users/jinym/Library/CloudStorage/OneDrive-개인/데이터L/selectstar/data/crops/{os.path.splitext(file)[0]}')

for dir in os.listdir(current):
    print(dir)
    os.system(f'cd {os.path.join(current, dir)} && rm -rf pdf_page {dir}.json && mv crop/* . && rm -rf crop')

SS0068
SS0067
SS0094
SS0146
.DS_Store
SS0115


mv: rename crop/* to ./*: No such file or directory
mv: rename crop/* to ./*: No such file or directory
sh: line 0: cd: /Users/jinym/Downloads/잡지_이미지제외/.DS_Store: Not a directory


SS0147
SS0139
SS0073
SS0074
SS0089
SS0088
SS0086
SS0065


mv: rename crop/* to ./*: No such file or directory


SS0091
SS0096
SS0097
SS0142
SS0143
SS0144
SS0102
SS0084
SS0070
SS0079
SS0082
SS0076


In [11]:
import pandas as pd
import os
import re
from collections import defaultdict

def extract_page_number(filename):
    """파일명에서 페이지 번호 추출"""
    # tb_0167_0001.png -> 0167, img_0158_0002.png -> 0158
    match = re.search(r'(?:tb_|img_)(\d{4})', filename)
    return match.group(1) if match else None

def organize_crop_files(crop_dir):
    """crop_dir의 폴더별로 파일들을 정리"""
    results = []
    
    for root, dirs, files in os.walk(crop_dir):
        # .DS_Store 파일 제외
        files = [f for f in files if f != '.DS_Store']
        
        if not files:  # 파일이 없으면 건너뛰기
            continue
        
        folder_name = os.path.basename(root)
        # if not folder_name:
            # continue
        
        # 파일들을 테이블과 이미지로 분류
        table_files = []
        image_files = []
        pages_with_files = set()
        
        for file in files:
            if file.startswith('tb_'):
                table_files.append(file)
                page_num = extract_page_number(file)
                if page_num:
                    pages_with_files.add(page_num)
            elif file.startswith('img_'):
                image_files.append(file)
                page_num = extract_page_number(file)
                if page_num:
                    pages_with_files.add(page_num)
        
        # 결과 저장
        results.append({
            '폴더명': folder_name,
            '테이블_파일_수': len(table_files),
            '테이블_파일_목록': ', '.join(sorted(table_files)),
            '이미지_파일_수': len(image_files),
            '이미지_파일_목록': ', '.join(sorted(image_files)),
            '파일이_있는_페이지_수': len(pages_with_files),
            '파일이_있는_페이지_목록': ', '.join(sorted(pages_with_files))
        })
    
    return results

# 실행
crop_dir = '/Users/jinym/Downloads/잡지_이미지제외/'
results = organize_crop_files(crop_dir)

# DataFrame 생성
df = pd.DataFrame(results)

# 결과 출력
print(f"총 {len(results)}개 폴더를 처리했습니다.")
print("\n처리 결과:")
print(df)

before_file = 'crop_files_magazine_BEFORE.xlsx'
after_file = 'crop_files_magazine_AFTER.xlsx'
df.to_excel(before_file, index=False)

총 25개 폴더를 처리했습니다.

처리 결과:
       폴더명  테이블_파일_수                                          테이블_파일_목록  \
0   SS0068        20  tb_0023_0001.png, tb_0024_0001.png, tb_0024_00...   
1   SS0067        23  tb_0016_0001.png, tb_0019_0001.png, tb_0023_00...   
2   SS0094        31  tb_0021_0001.png, tb_0040_0001.png, tb_0041_00...   
3   SS0146         8  tb_0008_0001.png, tb_0010_0001.png, tb_0014_00...   
4   SS0115        30  tb_0043_0001.png, tb_0078_0001.png, tb_0093_00...   
5   SS0147        16  tb_0008_0001.png, tb_0010_0001.png, tb_0012_00...   
6   SS0139         7  tb_0020_0001.png, tb_0029_0001.png, tb_0030_00...   
7   SS0073        20  tb_0012_0001.png, tb_0016_0001.png, tb_0043_00...   
8   SS0074        27  tb_0025_0001.png, tb_0053_0001.png, tb_0063_00...   
9   SS0089        29  tb_0019_0001.png, tb_0020_0001.png, tb_0033_00...   
10  SS0088        68  tb_0018_0001.png, tb_0024_0001.png, tb_0054_00...   
11  SS0086        49  tb_0026_0001.png, tb_0030_0001.png, tb_0031_00...   

In [12]:
df_before = pd.read_excel(before_file)

In [ ]:
set(df['폴더명']) - set(df_before['폴더명'])

In [ ]:
import pandas as pd
import re
from collections import defaultdict

def extract_page_numbers(file_list_str):
    """파일 목록 문자열에서 페이지 번호들을 추출"""
    if pd.isna(file_list_str) or file_list_str == '':
        return set()
    
    pages = set()
    files = file_list_str.split(', ')
    for file in files:
        # tb_0167_0001.png -> 0167, img_0158_0002.png -> 0158
        match = re.search(r'(?:tb_|img_)(\d{4})', file)
        if match:
            pages.add(match.group(1))
    return pages

def compare_analysis_files(before_file, after_file):
    """BEFORE와 AFTER 파일을 비교하여 차이점 분석"""
    
    # Excel 파일 읽기
    df_before = pd.read_excel(before_file)
    df_after = pd.read_excel(after_file)
    
    # 폴더명을 기준으로 매칭
    before_dict = {row['폴더명']: row for _, row in df_before.iterrows()}
    after_dict = {row['폴더명']: row for _, row in df_after.iterrows()}
    
    results = []
    
    for folder_name in before_dict.keys():
        if folder_name not in after_dict:
            # AFTER에 없는 폴더
            results.append({
                '폴더명': folder_name,
                '상태': 'AFTER에_없는_폴더',
                '테이블_파일_수': before_dict[folder_name]['테이블_파일_수'],
                '이미지_파일_수': before_dict[folder_name]['이미지_파일_수'],
                '없어진_테이블_파일': before_dict[folder_name]['테이블_파일_목록'],
                '없어진_이미지_파일': before_dict[folder_name]['이미지_파일_목록'],
                '없어진_페이지_수': before_dict[folder_name]['파일이_있는_페이지_수'],
                '없어진_페이지_목록': before_dict[folder_name]['파일이_있는_페이지_목록']
            })
        else:
            # 두 파일 모두에 있는 폴더 - 파일 비교
            before_row = before_dict[folder_name]
            after_row = after_dict[folder_name]
            
            # 파일 목록 비교
            before_table_files = set(before_row['테이블_파일_목록'].split(', ')) if pd.notna(before_row['테이블_파일_목록']) else set()
            after_table_files = set(after_row['테이블_파일_목록'].split(', ')) if pd.notna(after_row['테이블_파일_목록']) else set()
            
            before_image_files = set(before_row['이미지_파일_목록'].split(', ')) if pd.notna(before_row['이미지_파일_목록']) else set()
            after_image_files = set(after_row['이미지_파일_목록'].split(', ')) if pd.notna(after_row['이미지_파일_목록']) else set()
            
            # AFTER에 없는 파일들
            missing_table_files = before_table_files - after_table_files
            missing_image_files = before_image_files - after_image_files
            
            # 페이지 비교
            before_pages = set(before_row['파일이_있는_페이지_목록'].split(', '))
            after_pages = set(after_row['파일이_있는_페이지_목록'].split(', '))
            missing_pages = list(before_pages - after_pages)
            
            if missing_table_files or missing_image_files or missing_pages:
                results.append({
                    '폴더명': folder_name,
                    '상태': '일부_파일_누락',
                    '테이블_파일_수': before_row['테이블_파일_수'],
                    '이미지_파일_수': before_row['이미지_파일_수'],
                    '없어진_테이블_파일': ', '.join(sorted(missing_table_files)) if missing_table_files else '',
                    '없어진_이미지_파일': ', '.join(sorted(missing_image_files)) if missing_image_files else '',
                    '없어진_페이지_수': len(missing_pages),
                    '없어진_페이지_목록': ', '.join(sorted(missing_pages)) if missing_pages else '',
                    '최종_페이지_목록': ", ".join(sorted(after_pages))
                })
    
    return results

# 실행
before_file = 'crop_files_analysis.xlsx'
after_file = 'crop_files_analysis_AFTER.xlsx'

# 비교 분석 실행
comparison_results = compare_analysis_files(before_file, after_file)

# 결과를 DataFrame으로 변환
df_comparison = pd.DataFrame(comparison_results)

# 결과 출력
print(f"총 {len(comparison_results)}개 폴더에서 차이점을 발견했습니다.")
print("\n비교 결과:")
print(df_comparison)

# Excel 파일로 저장
df_comparison.to_excel('file_comparison_analysis.xlsx', index=False)

# 요약 통계
# total_missing_table_files = sum(len(row['없어진_테이블_파일'].split(', ')) if row['없어진_테이블_파일'] else 0 for row in comparison_results)
# total_missing_image_files = sum(len(row['없어진_이미지_파일'].split(', ')) if row['없어진_이미지_파일'] else 0 for row in comparison_results)
# total_missing_pages = sum(row['없어진_페이지_수'] for row in comparison_results)

# print(f"\n요약:")
# print(f"- 총 누락된 테이블 파일: {total_missing_table_files}개")
# print(f"- 총 누락된 이미지 파일: {total_missing_image_files}개")
# print(f"- 총 누락된 페이지: {total_missing_pages}개")